# 모델 파라미터 최적화


## 라이브러리 임포트 및 패션 MNIST 데이터셋 다운로드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os 

In [2]:
# 트레이닝 데이터셋을 다운로드한다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 테스트 데이터셋을 다운로드한다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## 모델 구성하기
훈련 모델을 작성한다.

In [3]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    # 포워드 패스    
    def forward(self, x):
        return self.linear_relu_stack(x)
    
model = DNN()
model

DNN(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=10, bias=True)
  )
)

## 하이퍼 파라미터, 손실 함수, 옵티마이저 설정하기

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimzer):
    size = len(dataloader.dataset)
    model.train() # 모델을 훈련 모드로 설정
    
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X) # 포워드 패스 수행
        loss = loss_fn(pred, y) # CE 연산
        
        optimzer.zero_grad() # 0 으로 초기화
        loss.backward() # 역전파하여 그래디언트 계산
        optimzer.step() # 연산된 그래디언트를 사용해 파라미터를 업데이트
        
        if batch % 100 == 0: # 매 100회차 마다 다음 내용 출력
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss}, [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval() # 모델을 실행 모드로 설정
    
    with torch.no_grad(): # 그래디언트 연산 안함
        for X, y in dataloader:
            pred = model(X) # 포워드 패스 수행
            test_loss += loss_fn(pred, y) # CE 연산
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 결과 일치하는지 확인
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n 정확도: {(100*correct):>0.1f}% 평균 Loss: {test_loss:>8f}\n')
    
for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print("Done!")
    

Epoch 1
--------------------------------------
loss: 2.296729564666748, [    0/60000]
loss: 2.2925941944122314, [ 6400/60000]
loss: 2.2674660682678223, [12800/60000]
loss: 2.26255464553833, [19200/60000]
loss: 2.255769968032837, [25600/60000]
loss: 2.215559244155884, [32000/60000]
loss: 2.2241554260253906, [38400/60000]
loss: 2.1885390281677246, [44800/60000]
loss: 2.192784309387207, [51200/60000]
loss: 2.155315399169922, [57600/60000]
Test Error: 
 정확도: 48.7% 평균 Loss: 2.156538

Epoch 2
--------------------------------------
loss: 2.163661479949951, [    0/60000]
loss: 2.1607232093811035, [ 6400/60000]
loss: 2.100435733795166, [12800/60000]
loss: 2.119097948074341, [19200/60000]
loss: 2.0714945793151855, [25600/60000]
loss: 2.004990816116333, [32000/60000]
loss: 2.0361452102661133, [38400/60000]
loss: 1.954427719116211, [44800/60000]
loss: 1.9725135564804077, [51200/60000]
loss: 1.8916754722595215, [57600/60000]
Test Error: 
 정확도: 56.0% 평균 Loss: 1.894535

Epoch 3
----------------------